# Тема “Работа с данными в Pandas”

**Задание 1**

Импортируйте библиотеку Pandas и дайте ей псевдоним pd. Создайте датафрейм authors со столбцами author_id и author_name, в которых соответственно содержатся данные: [1, 2, 3] и ['Тургенев', 'Чехов', 'Островский'].
Затем создайте датафрейм book cо столбцами author_id, book_title и price, в которых соответственно содержатся данные:  
[1, 1, 1, 2, 2, 3, 3],
['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
[450, 300, 350, 500, 450, 370, 290].


In [1]:
import pandas as pd

authors = pd.DataFrame({'author_id': [1, 2, 3], 'author_name': ['Тургенев', 'Чехов', 'Островский']},
                      columns = ['author_id', 'author_name'])
books = pd.DataFrame({'author_id': [1, 1, 1, 2, 2, 3, 3],
                   'book_title': ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 
                                  'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
                   'price': [450, 300, 350, 500, 450, 370, 290]})

**Задание 2**

Получите датафрейм authors_price, соединив датафреймы authors и books по полю author_id.


In [2]:
authors_price = pd.merge(authors, books, on = 'author_id', how = 'outer')
authors_price

,author_id,author_name,book_title,price
0,1,Тургенев,Отцы и дети,450
1,1,Тургенев,Рудин,300
2,1,Тургенев,Дворянское гнездо,350
3,2,Чехов,Толстый и тонкий,500
4,2,Чехов,Дама с собачкой,450
5,3,Островский,Гроза,370
6,3,Островский,Таланты и поклонники,290


**Задание 3**

Создайте датафрейм top5, в котором содержатся строки из authors_price с пятью самыми дорогими книгами.


In [9]:
top5 = authors_price.nlargest(5, 'price')
top5

,author_id,author_name,book_title,price
3,2,Чехов,Толстый и тонкий,500
0,1,Тургенев,Отцы и дети,450
4,2,Чехов,Дама с собачкой,450
5,3,Островский,Гроза,370
2,1,Тургенев,Дворянское гнездо,350


**Задание 4**

Создайте датафрейм authors_stat на основе информации из authors_price. В датафрейме authors_stat должны быть четыре столбца:
author_name, min_price, max_price и mean_price,
в которых должны содержаться соответственно имя автора, минимальная, максимальная и средняя цена на книги этого автора.


In [27]:
authors_stat = authors.copy()
authors_stat = authors_stat.drop('author_id', axis = 1)
df_min = pd.DataFrame(authors_price.groupby('author_name')['price'].min())
authors_stat = pd.merge(authors_stat, df_min, on = 'author_name', how = 'outer')
df_max = pd.DataFrame(authors_price.groupby('author_name')['price'].max())
authors_stat = pd.merge(authors_stat, df_max, on = 'author_name', how = 'outer')
df_mean = pd.DataFrame(round(authors_price.groupby('author_name')['price'].mean(), 2))
authors_stat = pd.merge(authors_stat, df_mean, on = 'author_name', how = 'outer')
authors_stat.columns = ['author_name', 'min_price', 'max_price', 'mean_price']
authors_stat

,author_name,min_price,max_price,mean_price
0,Тургенев,300,450,366.67
1,Чехов,450,500,475.00
2,Островский,290,370,330.00


**Задание 5**

Создайте новый столбец в датафрейме authors_price под названием cover, в нем будут располагаться данные о том, какая обложка у данной книги - твердая или мягкая. В этот столбец поместите данные из следующего списка:
['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'].
Просмотрите документацию по функции pd.pivot_table с помощью вопросительного знака.Для каждого автора посчитайте суммарную стоимость книг в твердой и мягкой обложке. Используйте для этого функцию pd.pivot_table. При этом столбцы должны называться "твердая" и "мягкая", а индексами должны быть фамилии авторов. Пропущенные значения стоимостей заполните нулями, при необходимости загрузите библиотеку Numpy.
Назовите полученный датасет book_info и сохраните его в формат pickle под названием "book_info.pkl". Затем загрузите из этого файла датафрейм и назовите его book_info2. Удостоверьтесь, что датафреймы book_info и book_info2 идентичны.


In [44]:
import numpy as np
authors_price['cover'] = ['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая']
book_info = pd.pivot_table(authors_price,
               index = ['author_name'],
               values = ['price'],
               columns = ['cover'],
               aggfunc = [np.sum],
               fill_value = 0)
book_info.to_pickle('book_info.pkl')
book_info2 = pd.read_pickle('book_info.pkl')
print(book_info)
print('-' * 27)
print(book_info2)

               sum        
             price        
cover       мягкая твердая
author_name               
Островский     660       0
Тургенев       650     450
Чехов            0     950
---------------------------
               sum        
             price        
cover       мягкая твердая
author_name               
Островский     660       0
Тургенев       650     450
Чехов            0     950
